In [329]:
import pathlib
import numpy as np
import pandas as pd
import json
import re
h = pathlib.Path.home()
pet_data = open(f"{h}/downloads/PET 2.txt",'r').readlines()

In [330]:
def petdata(pet_string):
    d = json.loads(pet_string)
    data_col = d['name'].replace(',','').replace('(','').replace(')','')
    values = np.array(d['data'])
    dates = values[:,0].astype(int)
    vals = values[:,1].astype(float)
    return data_col,dates,vals


In [331]:
def padd_arr(pet_string,vals_len):
    arr = petdata(pet_string)
    col_name = arr[0]
    vals = arr[2]
    if len(vals)>=vals_len:
        return col_name,vals[:vals_len]
    new_vals = np.pad(vals,(0,vals_len-len(vals)),'constant',constant_values=(0,np.nan))
    return col_name,new_vals

In [332]:
padd_arr(pet_data[700],1540)

('Midwest PADD 2 Imports of Crude Oil and Petroleum Products 4 Week Avg',
 array([2702., 2628., 2459., ...,   nan,   nan,   nan]))

In [333]:
from tqdm import tqdm_notebook
data_col,dates,vals = petdata(pet_data[0])
vals_len = len(vals)
df_pet = pd.DataFrame({'date':dates,data_col:vals})

for i in tqdm_notebook(range(1,len(pet_data))):
    try:
        data_col,vals = padd_arr(pet_data[i],vals_len)
    except:
        continue
    if 'weekly' not in data_col.lower():
        continue
    if '4 week avg' in data_col.lower():
        continue
    
    df_pet[data_col] = vals

df_pet


/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



,date,Florida Midgrade Conventional Retail Gasoline Prices Weekly,Boston MA Premium All Formulations Retail Gasoline Prices Weekly,San Francisco CA Premium Reformulated Retail Gasoline Prices Weekly,New York Harbor Premium Reformulated Retail Gasoline Prices Weekly,Houston TX Premium Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Premium Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Midgrade Reformulated Retail Gasoline Prices Weekly,Los Angeles CA Midgrade Reformulated Retail Gasoline Prices Weekly,San Francisco CA Midgrade Reformulated Retail Gasoline Prices Weekly,...,Rocky Mountain PADD 4 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,U.S. Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Midwest PADD 2 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Rocky Mountain PADD 4 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,U.S. Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly
0,20200817,2.395,2.631,3.578,2.632,2.425,3.028,2.639,3.286,3.415,...,0.0,0.0,48528.0,19625.0,5544.0,11841.0,0.0,11518.0,0.0,0.0
1,20200810,2.438,2.625,3.557,2.633,2.425,3.023,2.638,3.269,3.397,...,0.0,0.0,48324.0,19279.0,5128.0,11693.0,0.0,12224.0,0.0,0.0
2,20200803,2.469,2.630,3.581,2.646,2.438,3.034,2.651,3.284,3.414,...,0.0,0.0,47814.0,18463.0,5457.0,10957.0,0.0,12937.0,0.0,0.0
3,20200727,2.372,2.632,3.555,2.653,2.451,3.022,2.655,3.279,3.370,...,0.0,0.0,49471.0,19028.0,5494.0,11788.0,0.0,13161.0,0.0,0.0
4,20200720,2.386,2.636,3.508,2.667,2.458,3.019,2.662,3.260,3.313,...,0.0,0.0,49241.0,18567.0,5112.0,11822.0,0.0,13740.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20030623,1.569,1.692,2.100,1.696,1.561,1.891,1.749,1.868,2.004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,20030616,1.579,1.698,2.069,1.702,1.579,1.887,1.748,1.866,1.972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,20030609,1.550,1.683,1.983,1.710,1.571,1.833,1.666,1.799,1.878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,20030602,1.540,1.698,2.012,1.726,1.554,1.834,1.679,1.839,1.914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
df_pet.to_csv('./temp_folder/df_pet.csv',index=None)

In [335]:
lower_cols = [c.lower() for c in df_pet.columns.values]
df_pet_lower = df_pet.copy()
df_pet_lower.columns = lower_cols
df_pet_lower.to_csv('./temp_folder/df_pet_lower.csv',index=None)

In [336]:
cols = df_pet_lower.columns.values
df_reports = pd.DataFrame({'report':cols[np.where(cols!='date')]})
df_reports['colnum'] = df_reports.index.values
df_reports.to_csv('./temp_folder/df_pet_columns.csv',index=False)

In [337]:
df_reports[~df_reports.report.str.contains('stocks')]

,report,colnum
0,florida midgrade conventional retail gasoline prices weekly,0
1,boston ma premium all formulations retail gasoline prices weekly,1
2,san francisco ca premium reformulated retail gasoline prices weekly,2
3,new york harbor premium reformulated retail gasoline prices weekly,3
4,houston tx premium reformulated retail gasoline prices weekly,4
...,...,...
1109,east coast padd 1 imports of reformulated rbob with ether gasoline blending components weekly,1109
1110,midwest padd 2 rbob with ether gasoline blending components imports weekly,1110
1111,gulf coast padd 3 imports of reformulated rbob with ether gasoline blending components weekly,1111
1112,rocky mountain padd 4 rbob with ether gasoline blending components imports weekly,1112


In [338]:
reports_text = "\n".join(df_reports.report.values)


In [339]:
print(reports_text)

florida midgrade conventional retail gasoline prices weekly
boston ma premium all formulations retail gasoline prices weekly
san francisco ca premium reformulated retail gasoline prices weekly
new york harbor premium reformulated retail gasoline prices weekly
houston tx premium reformulated retail gasoline prices weekly
west coast padd 5 except california premium conventional retail gasoline prices weekly
west coast padd 5 except california midgrade reformulated retail gasoline prices weekly
los angeles ca midgrade reformulated retail gasoline prices weekly
san francisco ca midgrade reformulated retail gasoline prices weekly
new york harbor midgrade reformulated retail gasoline prices weekly
houston tx midgrade reformulated retail gasoline prices weekly
west coast padd 5 except california midgrade conventional retail gasoline prices weekly
seattle wa midgrade conventional retail gasoline prices weekly
seattle wa all grades conventional retail gasoline prices weekly
seattle wa regular c

In [340]:
import spacy
import pytextrank

# example text
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."
text = reports_text
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

# examine the top-ranked phrases in the document
for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
#     print(p.chunks)

0.1469    35  conventional retail gasoline prices
0.1446    87  retail gasoline prices
0.1440     2  reformulated retail gasoline prices
0.1415     5  regular conventional retail gasoline prices
0.1398     1  u.s. regular conventional retail gasoline prices
0.1388     4  regular reformulated retail gasoline prices
0.1381     1  u.s. premium conventional retail gasoline prices
0.1377     1  u.s. regular reformulated retail gasoline prices
0.1372     1  west coast regular conventional retail gasoline prices
0.1371     1  gulf coast regular conventional retail gasoline prices
0.1370     1  east coast regular conventional retail gasoline prices
0.1363     1  west coast midgrade conventional retail gasoline prices
0.1361     1  gulf coast midgrade conventional retail gasoline prices
0.1355     1  west coast premium conventional retail gasoline prices
0.1353     1  gulf coast premium conventional retail gasoline prices
0.1352     1  east coast premium conventional retail gasoline prices
0.13

In [341]:
ranks = []
counts = []
texts = []
for p in doc._.phrases:
    ranks.append(p.rank)
    counts.append(p.count)
    texts.append(p.text)



In [342]:
df_phrases_byrank = pd.DataFrame({'p_rank':ranks,'p_count':counts,'p_text':texts})
df_phrases_bycount = df_phrases_byrank.sort_values('p_count',ascending=False)


In [343]:
pd.set_option('display.max_colwidth', 0)
df_phrases_byrank

,p_rank,p_count,p_text
0,0.146925,35,conventional retail gasoline prices
1,0.144554,87,retail gasoline prices
2,0.144008,2,reformulated retail gasoline prices
3,0.141506,5,regular conventional retail gasoline prices
4,0.139849,1,u.s. regular conventional retail gasoline prices
...,...,...,...
477,0.008182,5,4 oxygenate
478,0.008130,1,3 gross
479,0.007795,7,2 rbob
480,0.007006,6,all countries


In [344]:
import dash_table
import dash
import dash_html_components as html

def run_reports(df,displayed_rows=100):

#     displayed_rows =10
    dt_reports = dash_table.DataTable(
        data=df.to_dict('records'),
        columns = [{'name':c,'id':c} for c in df.columns.values],
        page_size=displayed_rows,
        page_action='native',
        style_table={
            'overflowY':'auto',
#             'height': f'{35*(displayed_rows+1)+1}px'
        },
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'minWidth': '50px'
        },
        filter_action='native',
#         fixed_rows={'headers': True},
    )
    app = dash.Dash()
    s = {'display':'grid'}
    app.layout=html.Div(
        [dt_reports],
        style=s
    )
    app.run_server()
        

In [74]:
run_reports(df_phrases_bycount[['p_count','p_text']])

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 i

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET /_dash-layout HTTP/1.1" 200 -


In [345]:
df_phrases_byrank.to_csv(f'{h}/downloads/df_phrases_byrank.csv',index=False)

In [346]:

s = "padd.{1,2}[1-5][a-e]{0,1}"
df2 = df_phrases_byrank[df_phrases_byrank.p_text.str.contains(s)]
finds = [re.findall(s,v)[0] for v  in df2.p_text.values]
splits = [re.split(s,v) for v  in df2.p_text.values]
df2['token'] = finds
df2['left'] = [s[0] for s in splits]
df2['right'] = [None if len(s)<2 else s[1] for s in splits]

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [347]:
df2

,p_rank,p_count,p_text,token,left,right
58,0.122298,1,new england padd 1a premium conventional retail gasoline prices,padd 1a,new england,premium conventional retail gasoline prices
109,0.097362,1,central atlantic padd 1b premium conventional retail gasoline prices,padd 1b,central atlantic,premium conventional retail gasoline prices
128,0.088393,1,weekly rocky mountain padd 4 propane residential price weekly,padd 4,weekly rocky mountain,propane residential price weekly
130,0.087076,1,east coast padd 1 propane residential price weekly,padd 1,east coast,propane residential price weekly
140,0.081777,1,new england padd 1a propane residential price weekly,padd 1a,new england,propane residential price weekly
172,0.074039,1,west coast padd 5 commercial crude oil imports,padd 5,west coast,commercial crude oil imports
175,0.073314,1,east coast padd 1 commercial crude oil imports,padd 1,east coast,commercial crude oil imports
179,0.072295,1,gulf coast padd 3 commercial crude oil imports,padd 3,gulf coast,commercial crude oil imports
210,0.069362,1,east coast padd 1 propane wholesale/resale price weekly,padd 1,east coast,propane wholesale/resale price weekly
214,0.068660,1,east coast padd 1 refiner net production,padd 1,east coast,refiner net production


In [975]:
cols = ['california regular reformulated retail gasoline prices weekly','california regular reformulated retail gasoline prices']
df_pet2 = df_pet.copy()
df_pet2.columns = [v.lower()for v in df_pet2.columns.values]

### get padd info

In [350]:
cols = [c for c in df_pet.columns.values if len(re.findall('padd [0-9][a-z]{0,1}',c.lower()))>0]
cols = [c for c in cols if c!='date']
# cols = [c for c in cols if 'all' not in c.lower()]

In [351]:
cols

['West Coast PADD 5 Except California Premium Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Midgrade Reformulated Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Midgrade Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California No 2 Diesel Retail Prices Weekly',
 'West Coast PADD 5 Except California All Grades Reformulated Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California All Grades Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Regular Reformulated Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Regular Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Premium Reformulated Retail Gasoline Prices Weekly',
 'Lower Atlantic PADD 1C No 2 Diesel Ultra Low Sulfur 0-15 ppm Retail Prices Weekly',
 'West Coast PADD 5 Except California No 2 Diesel Ultra Low Sulfur 0-15 ppm Retail Prices Weekly',
 'Cent

In [352]:
df_pet[['date']+cols]

,date,West Coast PADD 5 Except California Premium Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Midgrade Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Midgrade Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California No 2 Diesel Retail Prices Weekly,West Coast PADD 5 Except California All Grades Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California All Grades Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Regular Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Regular Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Premium Reformulated Retail Gasoline Prices Weekly,...,Midwest PADD 2 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,Rocky Mountain PADD 4 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Midwest PADD 2 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Rocky Mountain PADD 4 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly
0,20200817,3.028,2.639,2.884,2.586,2.413,2.680,2.351,2.608,2.840,...,0.0,0.0,0.0,0.0,19625.0,5544.0,11841.0,0.0,11518.0,0.0
1,20200810,3.023,2.638,2.877,2.588,2.415,2.674,2.353,2.602,2.840,...,0.0,0.0,0.0,0.0,19279.0,5128.0,11693.0,0.0,12224.0,0.0
2,20200803,3.034,2.651,2.881,2.592,2.426,2.682,2.364,2.610,2.856,...,0.0,0.0,0.0,0.0,18463.0,5457.0,10957.0,0.0,12937.0,0.0
3,20200727,3.022,2.655,2.873,2.586,2.429,2.677,2.366,2.606,2.858,...,0.0,0.0,0.0,0.0,19028.0,5494.0,11788.0,0.0,13161.0,0.0
4,20200720,3.019,2.662,2.865,2.597,2.434,2.677,2.372,2.607,2.865,...,0.0,0.0,0.0,0.0,18567.0,5112.0,11822.0,0.0,13740.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20030623,1.891,1.749,1.767,NaN,1.688,1.705,1.650,1.665,1.850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,20030616,1.887,1.748,1.764,NaN,1.686,1.698,1.648,1.656,1.849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,20030609,1.833,1.666,1.706,NaN,1.607,1.641,1.570,1.600,1.765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,20030602,1.834,1.679,1.700,NaN,1.617,1.636,1.579,1.593,1.785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [353]:
def transform_cols(cols):
    padd_regex = "(padd [0-9][a-z]{0,1}){1,2}"
    padd_regions = []
    padds = []
    padd_categories = []
    padd_product_types = []
    for c in cols:
    #     padd = re.findall(padd_regex,c.lower())[0]
    #     split1 = re.split('padd [0-9][a-z]{0,1}',c.lower())
        split1 = re.split(padd_regex,c.lower())
        padd = ''.join(split1[1:-1])
        padd_region = split1[0].strip()
        split2 = split1[-1].strip().lower().split('of')
        padd_category = split2[0].strip()
        padd_product_type = '' if len(split2)<2 else split2[1].strip()
        padds.append(padd)
        padd_regions.append(padd_region)
        padd_categories.append(padd_category)
        padd_product_types.append(padd_product_type)

    df_padds = pd.DataFrame(
        {
            'padd':padds,
            'padd_region':padd_regions,
            'padd_category':padd_categories,
            'padd_product_type':padd_product_types,
            'col':cols
        }
    )
    return df_padds

In [1102]:
def create_unique_values(df_col_ids):
    df = df_col_ids[[c for c in df_col_ids.columns.values if c!='col']]
    # create a list of tuples, where each tuple has a column name and a list of unique values
    vals = [(c,sorted(list(df[c].unique()))) for c in df.dtypes[(df.dtypes=='object') | (df.dtypes=='str')].index]
    vals = [c for c in vals if (c is not None) and (len(c)>0)]
    # if there are no tuples in the list, return an empty DataFrame
    if len(vals)<1:
        return pd.DataFrame()
    # create a one row DataFrame with the first column, and the unique values for that column
    df_unique = pd.DataFrame({vals[0][0]:vals[0][1]})

    # Loop through the remaining columns, using pd. concat to add new columns
    #   to df_unique, which will automatically broadcast each column so that 
    #   all columns have the same length
    for v in vals[1:]:
        df_unique = pd.concat([df_unique,pd.DataFrame({v[0]:v[1]})],axis=1)
    df_unique = df_unique.fillna('')
    df_unique = df_unique[df_unique.apply(lambda r:len(''.join(r))>0,axis=1)]
    df_unique.index = list(range(len(df_unique)))
    return df_unique


In [1103]:
dft = pd.DataFrame({'a':['1','3','2','6'],'b':['a','b','f','d']})
create_unique_values(dft)

,a,b
0,1,a
1,2,b
2,3,d
3,6,f


In [1080]:
exp0 = '''
padd_regex = "(padd [0-9][a-z]{0,1}){1,2}"
split1 = re.split(padd_regex,c.lower())
padd = ''.join(split1[1:-1])
padd_region = split1[0].strip()
split2 = split1[-1].strip().lower().split('of')
padd_category = split2[0].strip()
padd_product_type = '' if len(split2)<2 else split2[1].strip()
print(padd)
print(padd_region)
print(padd_category)
print(padd_product_type)
'''

def padd_name(col):
    val = "(padd [0-9][a-z]{0,1}){1,2}"
    val = re.split(val,col.lower())
    val = ''.join(val[1:-1])
    return val
    
def padd_location(col):
    val = "(padd [1-5][a-z]{0,1}){1,2}"
    val = re.split(val,col.lower())[0]
    return val

def padd_region(col):
    val = "(padd [0-9][a-z]{0,1}){1,2}"
    val = re.split(val,col.lower())
    val = val[0].strip()
    return val

def padd_production_type(col):
    val = "(padd [0-9][a-z]{0,1}){1,2}"
    val = re.split(val,col.lower())
    val = val[-1].strip().lower().split('of')[0]
    return val
    
def padd_source(col):
    val = "(padd [0-9][a-z]{0,1}){1,2}"
    val = re.split(val,col.lower())
    val = val[-1].strip().lower().split('of')[-1]    
    return val

def padd_fuel_type(col):
    val = "(padd [1-5][a-z]{0,1}){1,2}"
    val = re.split(val,col.lower())
    val = re.split('of',val[-1])
    val = val[1]
    return val
    

def gas_price_region(col):
    val = re.split('price[ ]',col)
    val = [v.rstrip(' ') for v in val]
    val = ' '.join(val[0].split(' ')[0:-1])
    val = re.split('(regular)|(premium)|(midgrade)|(all grades)',val)[0]    
    return val

def gas_price_gas_type(col):
    val = re.split('price[ ]',col)
    val = [v.rstrip(' ') for v in val]
    val = ' '.join(val[0].split(' ')[0:-1])
    val = re.split('(regular)|(premium)|(midgrade)|(all grades)',val)[-1]
    val = re.split('retail',val)[0]
    return val

def gas_price_grade(col):
    val =  re.findall('(regular)|(premium)|(midgrade)|(all grades)',col) 
    if len(val)<1:
        return ''
    val = val[0]
    if type(val)==tuple:
        val = ''.join(list(val))
    return val

In [1178]:

def do_eval(code,cols):
    arr = []
    for col in cols:
        val = None
        try:
            val = code(col)
        except Exception as e:
#             val = str(e)
            val = ''
        arr.append(val.strip(' ').lstrip(' '))
    return arr

def eval_df(func_dict,cols):
    dict_df = {}
    for k in func_dict.keys():
        dict_df[k] = do_eval(func_dict[k],cols)
    df = pd.DataFrame(dict_df)
    return df


In [1179]:
def get_search_columns(df_pet_lower,column_extractor_list,isin_list,notin_list):
    column_extractor_dict = {}
    for m in column_extractor_list:
        column_extractor_dict[m.__name__] = m


    cols_no_date = [c for c in df_pet_lower.columns.values if c!='date']
    isin_regex = lambda c,TF,l:all([(len(re.findall(r,c))>0)==TF for r in l])
    cols_no_date = [c for c in cols_no_date if isin_regex(c,True,isin_list)]

    cols_no_date = [c for c in cols_no_date if isin_regex(c,False,notin_list)]

    df_eval = eval_df(column_extractor_dict,cols_no_date)
    df_eval['col'] = cols_no_date
    return df_eval

def get_search_lists(df_pet_lower,column_extractor_list,isin_list,notin_list):
    df_eval = get_search_columns(df_pet_lower,column_extractor_list,isin_list,notin_list)
    df_unique = create_unique_values(df_eval)
    return df_unique


In [1180]:
class PaddVolumeCategories():
    def __init__(self,df_pet_lower):
        column_extractor_list = [padd_name,padd_location,padd_production_type,padd_fuel_type]
        isin_list = ['(padd [1-5][a-z]{0,1}){1,2}','gasoline','of']
        notin_list = ['prices']
        self.df_cols = get_search_columns(df_pet_lower,column_extractor_list,isin_list,notin_list)
        self.df_categories = create_unique_values(self.df_cols)

class NonPaddGasPriceCategories():
    def __init__(self,df_pet_lower):
        column_extractor_list = [gas_price_region,gas_price_gas_type,gas_price_grade]
        isin_list = ['prices','gasoline','(regular)|(premium)|(midgrade)|(all grades)']
        notin_list = ['padd']
        self.df_cols = get_search_columns(df_pet_lower,column_extractor_list,isin_list,notin_list)
        self.df_categories = create_unique_values(self.df_cols)
        
        

In [1181]:
pvc = PaddVolumeCategories(df_pet_lower)

In [1182]:
pvc.df_categories

,padd_name,padd_location,padd_production_type,padd_fuel_type
0,padd 1,central atlantic,blender net production,conventional cbob gasoline blending components weekly
1,padd 1a,east coast,ending stocks,conventional gtab gasoline blending components weekly
2,padd 1b,gulf coast,imports,conventional motor gasoline ed55 and lower weekly
3,padd 1c,lower atlantic,imports from,conventional motor gasoline greater than ed55 weekly
4,padd 2,midwest,imports from all countries,conventional motor gasoline weekly
5,padd 3,new england,refiner and blender net input,conventional motor gasoline with fuel ethanol weekly
6,padd 4,rocky mountain,refiner and blender net production,conventional other gasoline blending components weekly
7,padd 5,west coast,refiner net production,finished conventional motor gasoline ed 55 and lower weekly
8,,,,finished conventional motor gasoline greater than ed 55 weekly
9,,,,finished conventional motor gasoline with ethanol weekly


In [1183]:
pvc.df_cols

,padd_name,padd_location,padd_production_type,padd_fuel_type,col
0,padd 2,midwest,imports,other reformulated motor gasoline weekly,midwest padd 2 imports of other reformulated motor gasoline weekly
1,padd 3,gulf coast,imports,other reformulated motor gasoline weekly,gulf coast padd 3 imports of other reformulated motor gasoline weekly
2,padd 4,rocky mountain,imports,other reformulated motor gasoline weekly,rocky mountain padd 4 imports of other reformulated motor gasoline weekly
3,padd 5,west coast,imports,other reformulated motor gasoline weekly,west coast padd 5 imports of other reformulated motor gasoline weekly
4,padd 1,east coast,imports from all countries,motor gasoline blending components rbob weekly,east coast padd 1 imports from all countries of motor gasoline blending components rbob weekly
...,...,...,...,...,...
245,padd 2,midwest,ending stocks,reformulated rbob with alcohol gasoline blending components weekly,midwest padd 2 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
246,padd 3,gulf coast,ending stocks,reformulated rbob with alcohol gasoline blending components weekly,gulf coast padd 3 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
247,padd 4,rocky mountain,ending stocks,reformulated rbob with alcohol gasoline blending components weekly,rocky mountain padd 4 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
248,padd 5,west coast,ending stocks,reformulated rbob with alcohol gasoline blending components weekly,west coast padd 5 ending stocks of reformulated rbob with alcohol gasoline blending components weekly


In [1184]:
npnc = NonPaddGasPriceCategories(df_pet_lower)
npnc.df_cols

,gas_price_region,gas_price_gas_type,gas_price_grade,col
0,florida,conventional,midgrade,florida midgrade conventional retail gasoline prices weekly
1,boston ma,all formulations,premium,boston ma premium all formulations retail gasoline prices weekly
2,san francisco ca,reformulated,premium,san francisco ca premium reformulated retail gasoline prices weekly
3,new york harbor,reformulated,premium,new york harbor premium reformulated retail gasoline prices weekly
4,houston tx,reformulated,premium,houston tx premium reformulated retail gasoline prices weekly
...,...,...,...,...
223,washington,all formulations,midgrade,washington midgrade all formulations retail gasoline prices weekly
224,u.s.,all formulations,premium,u.s. premium all formulations retail gasoline prices weekly
225,east coast,all formulations,premium,east coast premium all formulations retail gasoline prices weekly
226,midwest,all formulations,premium,midwest premium all formulations retail gasoline prices weekly


In [1185]:
df_all_categories = pd.concat((pvc.df_cols,npnc.df_cols)).fillna('')
df_all_categories = create_unique_values(df_all_categories)
df_all_categories

,padd_name,padd_location,padd_production_type,padd_fuel_type,gas_price_region,gas_price_gas_type,gas_price_grade
0,padd 1,central atlantic,blender net production,conventional cbob gasoline blending components weekly,boston ma,all formulations,all grades
1,padd 1a,east coast,ending stocks,conventional gtab gasoline blending components weekly,california,conventional,midgrade
2,padd 1b,gulf coast,imports,conventional motor gasoline ed55 and lower weekly,chicago,reformulated,premium
3,padd 1c,lower atlantic,imports from,conventional motor gasoline greater than ed55 weekly,chicago il,,regular
4,padd 2,midwest,imports from all countries,conventional motor gasoline weekly,cleveland oh,,
5,padd 3,new england,refiner and blender net input,conventional motor gasoline with fuel ethanol weekly,colorado,,
6,padd 4,rocky mountain,refiner and blender net production,conventional other gasoline blending components weekly,denver,,
7,padd 5,west coast,refiner net production,finished conventional motor gasoline ed 55 and lower weekly,denver co,,
8,,,,finished conventional motor gasoline greater than ed 55 weekly,east coast,,
9,,,,finished conventional motor gasoline with ethanol weekly,florida,,


In [1186]:
df_all_cols = pd.DataFrame({'col':df_pet_lower.columns.values})
df_all_cols = df_all_cols.merge(pvc.df_cols,on='col',how='left')
df_all_cols = df_all_cols.merge(npnc.df_cols,on='col',how='left').fillna('')
df_all_cols

,col,padd_name,padd_location,padd_production_type,padd_fuel_type,gas_price_region,gas_price_gas_type,gas_price_grade
0,date,,,,,,,
1,florida midgrade conventional retail gasoline prices weekly,,,,,florida,conventional,midgrade
2,boston ma premium all formulations retail gasoline prices weekly,,,,,boston ma,all formulations,premium
3,san francisco ca premium reformulated retail gasoline prices weekly,,,,,san francisco ca,reformulated,premium
4,new york harbor premium reformulated retail gasoline prices weekly,,,,,new york harbor,reformulated,premium
...,...,...,...,...,...,...,...,...
1128,gulf coast padd 3 ending stocks of reformulated rbob with alcohol gasoline blending components weekly,padd 3,gulf coast,ending stocks,reformulated rbob with alcohol gasoline blending components weekly,,,
1129,rocky mountain padd 4 ending stocks of reformulated rbob with alcohol gasoline blending components weekly,padd 4,rocky mountain,ending stocks,reformulated rbob with alcohol gasoline blending components weekly,,,
1130,west coast padd 5 ending stocks of reformulated rbob with alcohol gasoline blending components weekly,padd 5,west coast,ending stocks,reformulated rbob with alcohol gasoline blending components weekly,,,
1131,u.s. ending stocks of reformulated rbob with ether gasoline blending components weekly,,,,,,,


In [1195]:
c_padd_name = df_all_cols.padd_name.isin(['padd 1'])
c_padd_production_type = df_all_cols.padd_production_type.isin(['imports'])
c_padd_fuel_type = df_all_cols.padd_fuel_type.isin(['conventional cbob gasoline blending components weekly'])
c_all = c_padd_name & c_padd_production_type & c_padd_fuel_type
c_all = c_padd_name  & c_padd_fuel_type
df_all_cols[c_all]

,col,padd_name,padd_location,padd_production_type,padd_fuel_type,gas_price_region,gas_price_gas_type,gas_price_grade
565,east coast padd 1 imports of conventional cbob gasoline blending components weekly,padd 1,east coast,imports,conventional cbob gasoline blending components weekly,,,
656,east coast padd 1 refiner and blender net input of conventional cbob gasoline blending components weekly,padd 1,east coast,refiner and blender net input,conventional cbob gasoline blending components weekly,,,
662,east coast padd 1 ending stocks of conventional cbob gasoline blending components weekly,padd 1,east coast,ending stocks,conventional cbob gasoline blending components weekly,,,


In [1172]:
any(c_padd_production_type)

False

In [1138]:
def select_cols(s,cols):
    cols = [c for c in cols if c!='date']
    cols = [c for c in cols if len(re.findall(s,c.lower()))>0]
    return cols

In [1085]:
ss = ['ending stocks']
cols = df_pet_lower.columns.values
for s in ss:
    cols = select_cols(s,cols)
cols

['u.s. ending stocks of conventional motor gasoline greater than ed55 weekly',
 'east coast padd 1 ending stocks of conventional motor gasoline greater than ed55 weekly',
 'midwest padd 2 ending stocks of conventional motor gasoline greater than ed55 weekly',
 'gulf coast padd 3 ending stocks of conventional motor gasoline greater than ed55 weekly',
 'rocky mountain padd 4 ending stocks of conventional motor gasoline greater than ed55 weekly',
 'west coast padd 5 ending stocks of conventional motor gasoline greater than ed55 weekly',
 'u.s. ending stocks of conventional motor gasoline ed55 and lower weekly',
 'east coast padd 1 ending stocks of conventional motor gasoline ed55 and lower weekly',
 'midwest padd 2 ending stocks of conventional motor gasoline ed55 and lower weekly',
 'gulf coast padd 3 ending stocks of conventional motor gasoline ed55 and lower weekly',
 'rocky mountain padd 4 ending stocks of conventional motor gasoline ed55 and lower weekly',
 'west coast padd 5  ending

In [658]:
cols = [c for c in df_pet.columns.values if len(re.findall('padd [0-9][a-z]{0,1}',c.lower()))>0]
cols = [c for c in cols if c!='date']
cols = [c for c in cols if 'of' in c.lower()]


df_padds = transform_cols(cols)

In [659]:
df_padds.padd.unique()

array(['padd 2', 'padd 3', 'padd 4', 'padd 5', 'padd 1',
       'padd 4 and west coast padd 5', 'padd 1b', 'padd 1c', 'padd 2r',
       'padd 1a', 'padd 4 and padd 5'], dtype=object)

In [262]:
df_padds.padd_region.unique()

array(['rocky mountain', 'west coast', 'new england', 'central atlantic',
       'lower atlantic', 'east coast', 'midwest', 'gulf coast',
       'rocky mountains', ''], dtype=object)

In [257]:
df_padds.padd_category.unique()

array(['ending stocks', 'refiner and blender net production',
       'refiner net input', 'imports',
       'refiner blender and gas plant net production',
       'ending stocks excluding spr', 'refiner and blender net input',
       'efiner and blender net production', 'percent utilization',
       'refiner net production', 'blender net production',
       'oxygenate plant production', 'imports from  all countries',
       'imports from',
       'ending stocks excluding spr and including lease stock'],
      dtype=object)

In [843]:
PAD_REGEX = "(padd [0-9][a-z]{0,1}){1,2}"
PRODUCT_TYPE = "(kerosene-type jet fuel)|(gasoline)|(no 2[.]{0,1} diesel)|(propane( and propylene){0,1})|(crude)|(no. 2 heating oil)|(oxygenate)"
GRADE = "(regular)|(premium)|(midgrade)|(all grades)"
UNIT_TYPE = '(prices)|(inputs)|(net input)|(capacity)|(net production)|(production)|(imports)|(ending stocks)|(utilization)'
PROCESSOR_TYPE = '(refiner and blender)|(refiner)|(blender)|(oxygenate plant)|(gas plant)'
FORMULATION = "(all formulations)|(conventional)|(reformulated)"

def has_padd(col):
    return len(re.findall(PAD_REGEX,col.lower()))>0

def process_pad(cols,f,exclude_words=[]):
    vals = []
    for c in cols:
        if not has_padd(c):
            continue
        if any([ew in c for ew in exclude_words]):
            continue
        val = f(c)
        vals.append(val.strip().rstrip())
    return vals

def _padd(c):
    val = re.split(PAD_REGEX,c.lower())
    val = ''.join(val[1:-1])
    return val.strip().lstrip()

def get_padd(cols):
    return process_pad(cols,_padd)

def _padd_region(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[0].strip()
    g = GRADE
    val = re.split(g,val)[0]
    return val.strip().lstrip()

def get_padd_region(cols,**args):
    return process_pad(cols,_padd_region,**args)

def _pad_category(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[-1].strip().lower().split('of')[0]
    return val.strip().lstrip()

def get_padd_category(cols,**args):
    return process_pad(cols,_pad_category,**args)

def get_padd_category2(cols):
    vals = []
    for c in cols:
        if not has_padd(c):
            vals.append(None)
            continue
        val = re.split(PAD_REGEX,c.lower())
        val = val[-1].strip().lower().split('of')[0]
        if 'price' in val:
            vals.append(None)
            continue
        vals.append(val.strip().rstrip())
    return vals

def _pad_source(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[-1].strip().lower().split('of')[-1]
    return val.strip().lstrip()

def get_padd_source(cols,**args):
    return process_pad(cols,_pad_source,**args)


def get_padd_gasoline_type(cols,**args):
    vals = get_padd_source(cols,**args)
    vals = [re.split('gasoline',v)[0].strip().lstrip() for v in vals]
    return vals

def _padd_processor_type(c):
    val = re.split(PAD_REGEX,c.lower())
    val = ''.join(val[1:-1])
    return val.strip().lstrip()

def get_padd_processor_type(cols,**args):
    return process_pad(cols,_padd_processor_type,**args)

def get_ids(cols):
    def _findall(r,c):
        return ''.join([v for v in re.split(r,c.lower())[1:-1] if v is not None])
        l = re.findall(attribute_regexes[i],c)
        if l is None or len(l)<1:
            return ''
        if type(l[0]) == tuple:
            l = l[0]
        return ' '.join(l).strip().lstrip()
        
    all_attributes = {}
    attribute_regexes = [PAD_REGEX,PRODUCT_TYPE,GRADE,FORMULATION,UNIT_TYPE,PROCESSOR_TYPE]
    attribute_names = ['PAD_REGEX','PRODUCT_TYPE','GRADE','FORMULATION','UNIT_TYPE','PROCESSOR_TYPE']
    for i in range(len(attribute_regexes)):
        r = attribute_regexes[i]
        alist = [_findall(r,c) for c in cols]
        all_attributes[attribute_names[i]] = alist
    all_attributes['col'] = cols
    return pd.DataFrame(all_attributes)



In [786]:
set(get_padd_gasoline_type(df_pet_lower.columns.values))

{'',
 'all grades all formulations retail',
 'all grades conventional retail',
 'all grades reformulated retail',
 'commercial crude oil imports excluding spr weekly',
 'commercial kerosene-type jet fuel weekly',
 'conventional cbob',
 'conventional gtab',
 'conventional motor',
 'conventional other',
 'crude oil and petroleum products weekly',
 'crude oil weekly',
 'distillate fuel oil 0 to 15 ppm sulfur weekly',
 'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
 'distillate fuel oil greater than 2000 ppm sulfur weekly',
 'distillate fuel oil greater than 500 ppm sulfur weekly',
 'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
 'distillate fuel oil weekly',
 'except california all grades all formulations retail',
 'except california all grades conventional retail',
 'except california all grades reformulated retail',
 'except california midgrade all formulations retail',
 'except california midgrade conventional retail',
 'except california midgrade re

In [838]:
df_cols = get_ids(df_pet_lower.columns.values)
df_cols

,PAD_REGEX,PRODUCT_TYPE,GRADE,FORMULATION,UNIT_TYPE,PROCESSOR_TYPE,col
0,,,,,,,date
1,,gasoline,midgrade,conventional,prices,,florida midgrade conventional retail gasoline prices weekly
2,,gasoline,premium,all formulations,prices,,boston ma premium all formulations retail gasoline prices weekly
3,,gasoline,premium,reformulated,prices,,san francisco ca premium reformulated retail gasoline prices weekly
4,,gasoline,premium,reformulated,prices,,new york harbor premium reformulated retail gasoline prices weekly
...,...,...,...,...,...,...,...
1128,padd 3,gasoline,,reformulated,ending stocks,,gulf coast padd 3 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1129,padd 4,gasoline,,reformulated,ending stocks,,rocky mountain padd 4 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1130,padd 5,gasoline,,reformulated,ending stocks,,west coast padd 5 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1131,,gasoline,,reformulated,ending stocks,,u.s. ending stocks of reformulated rbob with ether gasoline blending components weekly


In [839]:
dfc = create_unique_values(df_cols)
dfc

,FORMULATION,GRADE,PAD_REGEX,PROCESSOR_TYPE,PRODUCT_TYPE,UNIT_TYPE
0,conventional,midgrade,padd 5,refiner blender and gas plant,gasoline,prices
1,all formulations,premium,padd 1c,refiner and blender,no 2 diesel,imports
2,reformulated,all grades,padd 1b,blender,propane,net production
3,,regular,padd 1a,refiner,propane and propylene and propylene,net input
4,,,padd 2,oxygenate plant,crude,ending stocks
5,,,padd 3,,oxygenate,production
6,,,padd 4,,no. 2 heating oil,utilization of refinery operable capacity
7,,,padd 1,,kerosene-type jet fuel,capacity
8,,,padd 4 and west coast padd 5,,gasoline non-oxygenate,inputs
9,,,padd 4 and padd 5,,,


In [824]:
padd_list = ['padd 1','padd 2','padd 3','padd 4','padd 5']
prod_type_list = ['gasoline']
grade_list = []#['regular']
unit_type_list = ['imports']#['ending stocks']
processor_type_list = []

all_true = pd.Series([True for _ in range(len(df_cols))])
do_screen = lambda c,l:all_true if len(l)<1 else df_cols[c].isin(l)

pad_regex_screen = do_screen('PAD_REGEX',padd_list)
pad_prod_type_screen = do_screen('PRODUCT_TYPE',prod_type_list)
pad_unit_type_screen = do_screen('UNIT_TYPE',unit_type_list)
pad_grade_screen = do_screen('GRADE',grade_list)
pad_processor_type_screen = do_screen('PROCESSOR_TYPE',processor_type_list)

screens = pad_regex_screen & pad_prod_type_screen & pad_unit_type_screen & pad_grade_screen & pad_processor_type_screen
cols_to_view = list(df_cols[screens].col.values)
cols_to_view = ['date'] + cols_to_view
df_screen = df_pet_lower[cols_to_view].fillna(0)
valid_cols = [c for c in df_screen.columns.values if df_screen[c].iloc[0:2].sum()>0]
df_screen[valid_cols]


,date,east coast padd 1 imports from all countries of motor gasoline blending components rbob weekly,west coast padd 5 imports of finished motor gasoline weekly,east coast padd 1 imports of finished motor gasoline weekly,east coast padd 1 imports of other conventional motor gasoline weekly,east coast padd 1 imports of conventional gtab gasoline blending components weekly,west coast padd 5 imports of conventional motor gasoline weekly,east coast padd 1 imports of conventional other gasoline blending components weekly,gulf coast padd 3 imports of conventional other gasoline blending components weekly,rocky mountain padd 4 conventional other gasoline blending components imports weekly,...,east coast padd 1 imports of total gasoline weekly,gulf coast padd 3 imports of total gasoline weekly,rocky mountain padd 4 imports of total gasoline weekly,west coast padd 5 imports of total gasoline weekly,east coast padd 1 imports of conventional cbob gasoline blending components weekly,east coast padd 1 imports of gasoline blending components weekly,gulf coast padd 3 imports of gasoline blending components weekly,rocky mountain padd 4 imports of gasoline blending components weekly,west coast padd 5 imports of gasoline blending components weekly,east coast padd 1 imports of reformulated rbob with alcohol gasoline blending components weekly
0,20200817,219.0,3.0,143.0,143.0,136.0,3.0,424.0,52.0,6.0,...,962.0,52.0,6.0,3.0,41.0,820.0,52.0,6.0,0.0,248.0
1,20200810,168.0,3.0,68.0,68.0,198.0,3.0,159.0,51.0,4.0,...,599.0,51.0,4.0,4.0,6.0,531.0,51.0,4.0,1.0,338.0
2,20200803,147.0,2.0,120.0,120.0,31.0,2.0,470.0,47.0,3.0,...,800.0,47.0,3.0,74.0,32.0,680.0,47.0,3.0,72.0,172.0
3,20200727,165.0,9.0,20.0,20.0,21.0,9.0,281.0,12.0,6.0,...,515.0,12.0,6.0,10.0,27.0,494.0,12.0,6.0,0.0,183.0
4,20200720,229.0,39.0,128.0,128.0,14.0,39.0,43.0,17.0,4.0,...,432.0,17.0,4.0,39.0,18.0,304.0,17.0,4.0,1.0,199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20030623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
896,20030616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
897,20030609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
898,20030602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [811]:
pad_regex_screen
type(pad_unit_type_screen)

pandas.core.series.Series

In [795]:
from dashapp import dashapp2 as dashapp
from plotly.offline import iplot

ys_per_graph = 4
clen = len(df_screen.columns.values)
glen = int(clen/ys_per_graph) + (1 if clen % ys_per_graph > 0 else 0)
all_y_cols = [c for c in df_screen.columns.values if c!='date']
for i in range(glen):
    b = i*ys_per_graph
    e = b + ys_per_graph
    cols = ['date'] + all_y_cols[b:e]
    iplot(dashapp.plotly_plot(df_screen[cols],x_column='date'))

In [736]:
glen

0

In [710]:
set(get_padd(df_pet_lower.columns.values))

{'padd 1',
 'padd 1a',
 'padd 1b',
 'padd 1c',
 'padd 2',
 'padd 2r',
 'padd 3',
 'padd 4',
 'padd 4 and padd 5',
 'padd 4 and west coast padd 5',
 'padd 5'}

In [478]:
set(get_padd_region(df_pet_lower.columns.values))

{'',
 'central atlantic',
 'east coast',
 'gulf coast',
 'lower atlantic',
 'midwest',
 'new england',
 'rocky mountain',
 'rocky mountains',
 'weekly gulf coast',
 'weekly rocky mountain',
 'west coast'}

In [490]:
ss =set(get_padd_source(df_pet_lower.columns.values,exclude_words=['price']))
len(ss),ss

(54,
 {'commercial crude oil imports excluding spr weekly',
  'commercial kerosene-type jet fuel weekly',
  'conventional cbob gasoline blending components weekly',
  'conventional gtab gasoline blending components weekly',
  'conventional motor gasoline ed55 and lower weekly',
  'conventional motor gasoline greater than ed55 weekly',
  'conventional motor gasoline weekly',
  'conventional motor gasoline with fuel ethanol weekly',
  'conventional other gasoline blending components imports weekly',
  'conventional other gasoline blending components weekly',
  'crude oil and petroleum products weekly',
  'crude oil weekly',
  'distillate fuel oil 0 to 15 ppm sulfur weekly',
  'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
  'distillate fuel oil greater than 2000 ppm sulfur weekly',
  'distillate fuel oil greater than 500 ppm sulfur weekly',
  'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
  'distillate fuel oil weekly',
  'finished conventional motor g

In [494]:
ss = set(get_padd_category(df_pet_lower.columns.values,exclude_words=['price']))
# ss = [s for s in ss if len(re.findall(unit_type,s))>0]#[s for s in ss if 'blender' in s]
len(ss),ss

(24,
 {'blender net production',
  'commercial crude oil imports excluding spr weekly',
  'conventional other gasoline blending components imports weekly',
  'efiner and blender net production',
  'ending stocks',
  'ending stocks excluding spr',
  'ending stocks excluding spr and including lease stock',
  'gross inputs into refineries weekly',
  'imports',
  'imports from',
  'imports from  all countries',
  'operable crude oil distillation capacity weekly',
  'oxygenate plant production',
  'percent utilization',
  'propane and propylene ending stocks excluding propylene at terminal weekly',
  'rbob with alcohol gasoline blending components imports weekly',
  'rbob with ether gasoline blending components ending stocks weekly',
  'rbob with ether gasoline blending components imports weekly',
  'refiner and blender net input',
  'refiner and blender net production',
  'refiner blender and gas plant net production',
  'refiner net input',
  'refiner net production',
  'reformulated gaso

In [501]:
ss = set(get_padd_gasoline_type(df_pet_lower.columns.values,exclude_words=['price']))
len(ss),ss

(42,
 {'',
  'commercial crude oil imports excluding spr weekly',
  'commercial kerosene-type jet fuel weekly',
  'conventional cbob',
  'conventional gtab',
  'conventional motor',
  'conventional other',
  'crude oil and petroleum products weekly',
  'crude oil weekly',
  'distillate fuel oil 0 to 15 ppm sulfur weekly',
  'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
  'distillate fuel oil greater than 2000 ppm sulfur weekly',
  'distillate fuel oil greater than 500 ppm sulfur weekly',
  'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
  'distillate fuel oil weekly',
  'finished conventional motor',
  'finished motor',
  'finished reformulated motor',
  'fuel ethanol weekly',
  'gross inputs into refineries weekly',
  'kerosene-type jet fuel weekly',
  'military kerosene-type jet fuel weekly',
  'motor',
  'operable crude oil distillation capacity weekly',
  'other conventional motor',
  'other finished conventional motor',
  'other finished reformu

In [508]:
ss = set(get_padd_category(df_pet_lower.columns.values,exclude_words=['price']))
ss = [re.split(UNIT_TYPE,s)[0] for s in ss]
len(ss),set(ss)

(24,
 {'',
  'blender ',
  'commercial crude oil ',
  'conventional other gasoline blending components ',
  'efiner and blender ',
  'gross ',
  'operable crude oil distillation ',
  'oxygenate plant ',
  'percent ',
  'propane and propylene ',
  'rbob with alcohol gasoline blending components ',
  'rbob with ether gasoline blending components ',
  'refiner ',
  'refiner and blender ',
  'refiner blender and gas plant ',
  'reformulated gasoline non-oxygenated '})

In [511]:
ss = set([c for c in df_pet_lower.columns.values if len(re.findall(PROCESSOR_TYPE,c))>0])
len(ss),ss

(211,
 {'central atlantic padd 1b refiner blender and gas plant net production of propane and propylene weekly',
  'east coast padd 1  refiner and blender net production of commercial kerosene-type jet fuel weekly',
  'east coast padd 1  refiner and blender net production of distillate fuel oil greater than 500 ppm sulfur weekly',
  'east coast padd 1  refiner and blender net production of military kerosene-type jet fuel weekly',
  'east coast padd 1 blender net production of finished motor gasoline weekly',
  'east coast padd 1 gross inputs into refineries weekly',
  'east coast padd 1 oxygenate plant production of fuel ethanol weekly',
  'east coast padd 1 percent utilization of refinery operable capacity weekly',
  'east coast padd 1 refiner and blender net input of conventional cbob gasoline blending components weekly',
  'east coast padd 1 refiner and blender net input of conventional gtab gasoline blending components weekly',
  'east coast padd 1 refiner and blender net input of 